In [1]:
import sys
import os
import pprint
pp = pprint.PrettyPrinter(indent=2, sort_dicts=False)

In [2]:
## IMPORT KERIPY 
import warnings
warnings.filterwarnings("ignore")

base_path = os.getcwd()
relative_path = os.path.abspath(os.path.join(base_path, '../'))
sys.path.append(relative_path)

from src.keri.core import coring
from src import keri
keri.__version__

'1.2.0-rc1'

In [3]:
print('/'.join(keri.core.coring.__file__.split('/')[6:]))

my_forks/keripy/src/keri/core/coring.py


In [4]:
!git branch

  compact-acdc-algo
* compact-acdc-algo-test
  main


In [5]:
## load schema tests

import json
import os

def load_json(fp, root_path = ''):
    if root_path != '' and root_path[-1] != '/':
        fp = root_path + '/' + fp
    else:
        fp = root_path + fp
    with open(fp) as jsf:
        
        return json.load(jsf)

def get_files(_path):
     try:
        return [f for f in os.listdir(_path) if os.path.isfile(os.path.join(_path, f))]

     except FileNotFoundError:
        print("The specified directory does not exist.")
        return []





In [6]:
schema_path = '../tests/data/schemas'
schema_files = get_files(schema_path)
schema_files

['legal-entity-official-organizational-role-vLEI-credential.json',
 'ecr-authorization-vlei-credential.json',
 'oor-authorization-vlei-credential.json',
 'legal-entity-engagement-context-role-vLEI-credential.json',
 'verifiable-ixbrl-report-attestation.json',
 'legal-entity-vLEI-credential.json',
 'qualified-vLEI-issuer-vLEI-credential.json']

In [7]:
acdc_path = '../tests/data/acdcs'
acdc_files = get_files(acdc_path)
acdc_files

['legal-entity-official-organizational-role-vLEI-credential.json',
 'ecr-authorization-vlei-credential.json',
 'oor-authorization-vlei-credential.json',
 'verifiable-ixbrl-report-attestation.json',
 'legal-entity-engagement-context-role-vLEI-credential-le.json',
 'legal-entity-vLEI-credential.json',
 'qualified-vLEI-issuer-vLEI-credential.json',
 'legal-entity-engagement-context-role-vLEI-credential-said.json',
 'legal-entity-engagement-context-role-vLEI-credential-auth.json']

In [8]:
_files = {'acdcs': acdc_files, 'schemas': schema_files}

In [9]:
!git branch

  compact-acdc-algo
* compact-acdc-algo-test
  main


In [10]:
_files

{'acdcs': ['legal-entity-official-organizational-role-vLEI-credential.json',
  'ecr-authorization-vlei-credential.json',
  'oor-authorization-vlei-credential.json',
  'verifiable-ixbrl-report-attestation.json',
  'legal-entity-engagement-context-role-vLEI-credential-le.json',
  'legal-entity-vLEI-credential.json',
  'qualified-vLEI-issuer-vLEI-credential.json',
  'legal-entity-engagement-context-role-vLEI-credential-said.json',
  'legal-entity-engagement-context-role-vLEI-credential-auth.json'],
 'schemas': ['legal-entity-official-organizational-role-vLEI-credential.json',
  'ecr-authorization-vlei-credential.json',
  'oor-authorization-vlei-credential.json',
  'legal-entity-engagement-context-role-vLEI-credential.json',
  'verifiable-ixbrl-report-attestation.json',
  'legal-entity-vLEI-credential.json',
  'qualified-vLEI-issuer-vLEI-credential.json']}

In [11]:
def _log(message, file_path="test_log.txt", _print=False):
    """
    Logs a message to the console and appends it to a log file.

    Parameters:
        message (str): The message to log.
        file_path (str): The path to the log file.
    """
    # Print the message to the console
    if _print:
        print(message)
    
    # Append the message to the log file
    with open(file_path, "a") as log_file:
        log_file.write(message + "\n")
files = []
_path = '../tests/data/'
for kind in _files:
    # print(kind)
    this_path = _path + kind + '/'
    # continue
    for f in _files[kind]:
        # print(this_path+f)
        
        this = {'file_path': this_path+f, 'file_name': f}
        this['data'] = load_json(f,this_path)
        this['kind'] = kind
        files.append(this)
    # print(files[f]['data'] )
    # break
#     # files
for f in files:
    # print(f.keys())
    # print(f['file_name'])
    label = 'd' if f['kind'] == 'acdcs' else '$id'
    f['label'] = label
    f['said_paths'] = coring.Saider._map_paths_to_label(f['data'], label)
    f['saidified'] = coring.Saider._saidify(f['data'], label=label)
    # break
report = ''
false_matches = []
for f in files:
    
    _log(f"{'='*88}")
    _log( f"file: {f['file_name']}")
    _log(f"{'-'*88}")
    _log(f['kind'])
    _log(f"{str(list(f.keys()))}")
    _log( f"file: {f['file_name']}")
    _log(f"{'-'*30} DATA {'-'*30}")
    js = json.dumps(f['data'], indent=2)
    _log(js)
    # pp.pprint(f['data'])
    _log(f"{'='*66}")
    
    _log(f"{'-'*27} SAIDIFIED {'-'*27}")
    _log(f"{'-'*29} PATHS {'-'*29}")
    _log(json.dumps(f['said_paths']))#, indent=2))
    _log(f"{'-'*29}  SADS {'-'*29}")
    for k in f['saidified']['sads']:
        _sad = f['saidified']['sads']
        this_said = coring.Saider.saidify(_sad[k], label=f['label'])[0].qb64
        _log('*'*88)
        manual_match = this_said == _sad[k][f['label']]
        if not manual_match:
            false_matches.append(manual_match)
        _log(f"{k}:\tMatch: {manual_match}\t{this_said}")
        _log(json.dumps(_sad[k], indent=2))
    _log('*'*88)
    _log(f"{'-'*27} COMPACTED {'-'*27}")
    js = json.dumps(f['saidified']['compact'], indent=2)
    _log(js)
    _log(f"{'-'*25} NON_COMPACTED {'-'*25}")
    js = json.dumps(f['saidified']['non_compact'], indent=2)
    _log(js)
    # pp.pprint(f['saidified'])
    # break
false_matches

[]